# AM_AdviesScraper

Doel: AM adviezen uitgebracht vanaf 2019 tot en met 2024 verzamelen.

Toepassing: Bachelor-scriptie Informatiekunde, Universiteit van Amsterdam

Contact: mitchell.malaihollo@student.uva.nl

Metadata Formatering volgens de [Woogle metadata specificatie](https://github.com/wooverheid/WoogleDocumentatie/tree/master/SPEC%20MetadataSchema) 

In [1]:
import time
from urllib.parse import urljoin
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from openai import OpenAI
from dotenv import load_dotenv
import json
from selenium.common.exceptions import NoSuchElementException
from urllib.parse import urljoin, urlparse

In [2]:
# Aantal pagina's aanpassen

HREF_ID_WOORDEN = {"briefadvies", "wetsadvies", "advies", "adviesrapport", "adviesproject"}
ID_WOORDEN = {"briefadvies", "advies:", "advies", "adviesrapport", "adviesproject", "wetsadvies"}

USER_AGENT = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:124.0) "
    "Gecko/20100101 Firefox/124.0 "
    "AdviescollegeScraper (+mailto:14605120@student.uva.nl; "
    "voor BSc-scriptie Informatiekunde UvA; "
    "doel: informatie inwinnen over verschillende adviescolleges)"
)

BASE_URL = (
    "https://www.adviesraadmigratie.nl/documenten"
    "?filters%5B0%5D%5Bfield%5D=publication_date"
    "&filters%5B0%5D%5Bvalues%5D%5B0%5D%5Bfrom%5D=2019-01-01T00%3A00%3A00%2B01%3A00"
    "&filters%5B0%5D%5Bvalues%5D%5B0%5D%5Bname%5D=specificPeriodFrom"
    "&filters%5B0%5D%5Bvalues%5D%5B1%5D%5Bto%5D=2024-12-31T23%3A59%3A59%2B01%3A00"
    "&filters%5B0%5D%5Bvalues%5D%5B1%5D%5Bname%5D=specificPeriodTo"
    "&filters%5B0%5D%5Btype%5D=all"
)

BASE = "https://www.adviesraadmigratie.nl"

env_path = r"C:\Users\31633\OneDrive\Bureaublad\OpenAI_API_Key.env"  
load_dotenv(env_path)
api_key = os.getenv("OPENAI_API_KEY")
print("Key geladen:", api_key is not None)

Key geladen: True


In [3]:
import re

def beginSelenium():
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument(f"user-agent={USER_AGENT}")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=options
    )
    return driver

def aantalPaginas():
    x = driver.find_element(
        By.CSS_SELECTOR,
        "#main-content > div.search-block > div.search-block__results > nav > ul > li:nth-child(7) > button > span.sr-only"
    )
    y = x.text.strip()
    pagina = re.sub(r"\D", "", y)
    return int(pagina)



driver = beginSelenium()
driver.get(BASE_URL)
time.sleep(2)
print("Maximaal aantal pagina's om te controleren: " + str(aantalPaginas()))
print("-----------------------")

adviesURLS = []
for paginaNummer in range(1, aantalPaginas() + 1):
    if paginaNummer == 1:
        BASE_paginaURL = BASE_URL
    else:
        BASE_paginaURL = f"{BASE_URL}&current=n_{paginaNummer}_n"

    print("Pagina: " + str(paginaNummer))
    print(BASE_paginaURL)

    driver.get(BASE_paginaURL)
    time.sleep(2)

    li_items = driver.find_elements(
        By.CSS_SELECTOR,
        "ul.sui-results-container.links-list li.links-list__item"
    )

    for li in li_items:
        meta_spans = li.find_elements(
            By.CSS_SELECTOR,
            ".meta-data.link-plus__meta-data span"
        )

        # metadata type
        doc_type = meta_spans[0].text.strip().lower()
        a_tag = li.find_element(By.CSS_SELECTOR, "a.link-plus")
        href = a_tag.get_attribute("href")
        if href.startswith("/"):
            href = BASE + href

        # eerste woord van de href
        href_delen = urlparse(href).path
        href_deel = href_delen.strip("/").split("/")[-1]
        eerste_woord = href_deel.split("-")[0]

        # eerste woord van de titel
        title_element = li.find_element(By.CSS_SELECTOR, "h3.link-plus__title")
        title_text = title_element.text.strip().lower()
        eerste_woord_titel = title_text.split()[0]
        
        # metadata check
        if "advies" in doc_type:
            print(href)
            print("dit is een advies")
            print("-")
            adviesURLS.append(href)
            continue

        # href eerste woord check
        if eerste_woord in HREF_ID_WOORDEN:
            print(href)
            print("dit is een advies")
            print("-")
            adviesURLS.append(href)
            continue

        # titel check dus Advies:
        if eerste_woord_titel in ID_WOORDEN:
            print(href)
            print("dit is een advies")
            print("-")
            adviesURLS.append(href)
            continue
    time.sleep(5)

    print("-------------------")

Maximaal aantal pagina's om te controleren: 13
-----------------------
Pagina: 1
https://www.adviesraadmigratie.nl/documenten?filters%5B0%5D%5Bfield%5D=publication_date&filters%5B0%5D%5Bvalues%5D%5B0%5D%5Bfrom%5D=2019-01-01T00%3A00%3A00%2B01%3A00&filters%5B0%5D%5Bvalues%5D%5B0%5D%5Bname%5D=specificPeriodFrom&filters%5B0%5D%5Bvalues%5D%5B1%5D%5Bto%5D=2024-12-31T23%3A59%3A59%2B01%3A00&filters%5B0%5D%5Bvalues%5D%5B1%5D%5Bname%5D=specificPeriodTo&filters%5B0%5D%5Btype%5D=all
https://www.adviesraadmigratie.nl/documenten/2024/09/10/adviesrapport-afgewogen-arbeidsmigratie
dit is een advies
-
https://www.adviesraadmigratie.nl/documenten/2024/09/06/advies-wet-versterking-waarborgfunctie-awb
dit is een advies
-
https://www.adviesraadmigratie.nl/documenten/2024/06/14/advies-over-wetsvoorstel-invoering-schorsende-werking-in-dublinzaken
dit is een advies
-
-------------------
Pagina: 2
https://www.adviesraadmigratie.nl/documenten?filters%5B0%5D%5Bfield%5D=publication_date&filters%5B0%5D%5Bvalues%5D

In [4]:
print("\nAlle unieke adviezen links:")
for link in adviesURLS:
    print(link)

print(f"\nTotaal aantal unieke adviezen-links: {len(adviesURLS)}")


Alle unieke adviezen links:
https://www.adviesraadmigratie.nl/documenten/2024/09/10/adviesrapport-afgewogen-arbeidsmigratie
https://www.adviesraadmigratie.nl/documenten/2024/09/06/advies-wet-versterking-waarborgfunctie-awb
https://www.adviesraadmigratie.nl/documenten/2024/06/14/advies-over-wetsvoorstel-invoering-schorsende-werking-in-dublinzaken
https://www.adviesraadmigratie.nl/documenten/2024/03/13/adviesrapport-geen-derderangsburgers.-de-risicos-voor-gedetacheerde-arbeidsmigranten-en-de-nederlandse-samenleving
https://www.adviesraadmigratie.nl/documenten/2023/09/27/advies-voor-de-nota-van-wijziging-bij-de-novelle-bij-het-wetsvoorstel-wet-terugkeer-en-vreemdelingenbewaring
https://www.adviesraadmigratie.nl/documenten/2023/07/17/wetsadvies-besluit-en-regeling-ter-uitvoering-van-de-wet-gemeentelijke-taak-mogelijk-maken-asielopvangvoorzieningen
https://www.adviesraadmigratie.nl/documenten/2023/07/11/wetsadvies-wet-testen-bij-vertrek
https://www.adviesraadmigratie.nl/documenten/2023/05/

In [5]:
res = []
for url in adviesURLS:
    client = OpenAI()

    prompt = f"""
    Lees dit document: {url}

    Vul de onderstaande JSON in volgens https://github.com/wooverheid/WoogleDocumentatie/tree/master/SPEC%20MetadataSchema:

    {{
        "dc_title": "",
        "foi_publishedDate": "",
        "dc_description": ""
    }}

    Regels:
    - Alleen écht gevonden info invullen.
    - Max 100 woorden in dc_description.
    - Ontbreekt iets? dan dat veld "" laten.
    - datum in jjjj-mm-dd van het daadwerkelijke gepubliceerde advies.
    - Output = alleen geldige JSON niks anders.
    """
    response = client.responses.create(
        model="gpt-5.1",
        input=prompt,
        tools=[{"type": "web_search"}],
        max_output_tokens=800,
    )
    
    json_str = response.output_text.strip()

    try:
        data = json.loads(json_str)
    except json.JSONDecodeError:
        print("Kon JSON niet parsen voor URL:", url)
        print("Ruwe output was:\n", json_str)
        continue

    data["dc_source"] = url
    data["dc_publisher"] = "oorg10252"
    data["dc_type"] = "2e"
        
    res.append(data)
    
    print(json.dumps(data, ensure_ascii=False, indent=2))
    
    time.sleep(12)

with open("AM_adviesScraper_data1.json", "w", encoding="utf-8") as f:
    json.dump(res, f, ensure_ascii=False, indent=2)
    
print("\nAlle JSON-data opgeslagen in AM_adviesScraper_data1.json")

{
  "dc_title": "Advies ‘Afgewogen arbeidsmigratie. Gericht arbeidsmigratiebeleid voor brede welvaart'",
  "foi_publishedDate": "2024-09-10",
  "dc_description": "De Adviesraad Migratie pleit in dit adviesrapport voor een arbeidsmigratiebeleid dat niet alleen uitgaat van economische groei, maar ook van maatschappelijke en ecologische effecten, in lijn met het concept brede welvaart. Het rapport schetst hoe een brede‑welvaartsbenadering van arbeidsmigratie kan worden uitgewerkt, met onder meer bandbreedtes voor het aantal arbeidsmigranten en een brede welvaartstoets voor toelating van arbeidsmigranten van buiten de EU.",
  "dc_source": "https://www.adviesraadmigratie.nl/documenten/2024/09/10/adviesrapport-afgewogen-arbeidsmigratie",
  "dc_publisher": "oorg10252",
  "dc_type": "2e"
}
{
  "dc_title": "Advies Wet versterking waarborgfunctie Awb",
  "foi_publishedDate": "2024-07-31",
  "dc_description": "Advies van de Adviesraad Migratie over de voorgestelde aanpassingen voor de Wet verster

In [6]:
with open("AM_adviesScraper_data1.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print("Aantal items in JSON bestand: " + str(len(data)))

Aantal items in JSON bestand: 44


In [7]:
res2 = []

with open("AM_adviesScraper_data1.json", "r", encoding="utf-8") as f:
    data = json.load(f)

def retrieveLinksPagina2(driver, urltje):
    resje = {}
    driver.get(urltje)

    # Alle <li> elementen ophalen
    li_elements = driver.find_elements(By.CSS_SELECTOR, "ul.download-list li.download-list__item")
    
    for li in li_elements:
        # Titel ophalen
        title = li.find_element(By.CSS_SELECTOR, "span.title").text.strip()
        
        # Download link ophalen
        url = li.find_element(By.CSS_SELECTOR, "a.download-list__link").get_attribute("href")
    
        # In dictionary zetten
        resje[title] = url
        
    return resje    

driver = beginSelenium()

for item in data:
    dc_source2 = item.get("dc_source")

    urltjes = retrieveLinksPagina2(driver, dc_source2)

    foi_files = []
    for key, value in urltjes.items():
        if not value:
            continue
        foi_files.append({
            "foi_title": key,
            "foi_source": value,
        })

    res2.append({
        "dc_title": item.get("dc_title"),
        "foi_publishedDate": item.get("foi_publishedDate"),
        "dc_description": item.get("dc_description"),
        "dc_source": dc_source2,
        "dc_publisher": item.get("dc_publisher"),
        "dc_type": item.get("dc_type"),
        "foi_files": foi_files
    })

    time.sleep(5)

driver.quit()

with open("AM_adviesScraper_data2.json", "w", encoding="utf-8") as f:
    json.dump(res2, f, ensure_ascii=False, indent=2)

print("\nAlle JSON-data opgeslagen in AM_adviesScraper_data2.json")


Alle JSON-data opgeslagen in AM_adviesScraper_data2.json


In [8]:
with open("AM_adviesScraper_data2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print("Aantal items in JSON bestand: " + str(len(data)))

Aantal items in JSON bestand: 44
